In [24]:
from context import BerryFluxDiag as bfd
from pymatgen.core import Structure
import numpy as np

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Step 0: Need pymatgen for polar and non-polar reference cells that have already been relaxed and have the same number of atoms in the same order 

# Step 1: Set up polar and non polar structures

In [13]:
material = 'BaTiO3'
mp_id = 'mp-5986'
base_path = '../io_files/VASP_io/'
pol_POSCAR_file = base_path+f'poscar_p_relaxed_{mp_id}_{material}.vasp'
np_POSCAR_file = base_path+f'poscar_np_relaxed_{mp_id}_{material}.vasp'

pol_orig_struct = Structure.from_file(pol_POSCAR_file)
np_orig_struct = Structure.from_file(np_POSCAR_file)

# Step 2: Find the translation between the polar and non-polar structures that minimizes the maximal atomic displacement between any single ion

In [19]:
np_trans_struct = bfd.preprocess.translate_structs(pol_orig_struct, np_orig_struct)

# Step 3: If the maximal atomic displacement is large (greater than the 0.3 A default), interpolate the two structures so that no displacements between successive structs are larger than 0.3 A

In [25]:
# compute how many interpolations are necessary
max_disp_structs = bfd.preprocess.calc_max_disp(pol_orig_struct, np_trans_struct, trans=None)
MAX_DISP_THRESH = 0.3
num_interps = int(np.ceil(max_disp_structs/MAX_DISP_THRESH))

In [32]:
# structs[0] is polar, structs [-1] is translated non-polar, intermediate list values are interpolated Structures
structs = pol_orig_struct.interpolate(np_trans_struct, num_interps, interpolate_lattices=True)

# Step 4: Write Structures and electronic structure input
For all ES calcs:
- symmetry turned off (that is, compute wavefunctions for the full Brillouin zone, not just the irreducible Brillouin zone)
- save full wavefunction files
- k-point grid must be the same for all calculations across all structures

# Step 4a: VASP input
- key tags are ISYM=-1, LWAVE=TRUE

# Step 4b: Quantum ESPRESSO input
- use ONCV pseudopotentials (or other norm-conserving pseudopotentials)
- key tags in &SYSTEM are nosym = .TRUE. and noinv = .TRUE.